In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches

In [3]:
def tf_reset():
    try:
        sess.close()
    except:
        pass
    tf.reset_default_graph()
    return tf.Session()

### build network

In [81]:
def build_mlp(input_placeholder, output_size, scope, n_layers, size, activation=tf.tanh, output_activation=None):
    """
        Builds a feedforward neural network
        
        arguments:
            input_placeholder: placeholder variable for the state (batch_size, input_size)
            output_size: size of the output layer
            scope: variable scope of the network
            n_layers: number of hidden layers
            size: dimension of the hidden layer
            activation: activation of the hidden layers
            output_activation: activation of the ouput layers

        returns:
            output placeholder of the network (the result of a forward pass) 

        Hint: use tf.layers.dense    
    """
    # YOUR CODE HERE
    fwd = input_placeholder
    for _ in range(n_layers):
        fwd = tf.layers.dense(fwd, size, activation=activation)
    output_placeholder = tf.layers.dense(fwd, output_size, activation=output_activation)
    return output_placeholder

In [122]:
input_ph = tf.placeholder(dtype=tf.float32, shape = [None, 2])
output_ph = build_mlp(input_ph, 3, None, 2, 5, tf.nn.relu, tf.nn.softmax)
sess.run(tf.global_variables_initializer())
output_run = sess.run(output_ph, feed_dict={input_ph: np.array([[2,3], [4,5]])})

In [85]:
print(output_run)

[[0.37970188 0.02442171 0.59587634]
 [0.3143339  0.00279099 0.68287504]]


### matrix broadcasting

In [89]:
sess = tf_reset()
b = tf.constant([[1,1,1], [1,1,1]], dtype = tf.float32)
a = tf.constant([[2],[3]], dtype = tf.float32)
c = b * a
print(sess.run(c))

[[2. 2. 2.]
 [3. 3. 3.]]


### random sampling

In [151]:
sess = tf_reset()
logits = tf.constant([[0,0,10], [10,0,0]], dtype = tf.float32)
samples = tf.random.categorical(tf.log(logits), 1)
print(sess.run(samples))

[[2]
 [0]]


In [153]:
import tensorflow_probability as tfp
labels = tf.constant([[1,0]], dtype = tf.int32)
dist = tfp.distributions.Categorical(logits)
print(sess.run(dist.log_prob(tf.reshape(labels, [-1]))))

[-1.0000091e+01 -9.0833353e-05]


In [117]:
1.0 / (np.exp(10)+2)

4.539580782951091e-05

In [140]:
sess = tf_reset()
mu = tf.constant([[0, 0, 0], [1 ,2 ,3 ]], dtype = tf.float32)
std = tf.constant([[.1, .1, 10]], dtype = tf.float32)
samples = tf.random.normal(tf.shape(mu))
samples = mu + std * samples
samples_run = sess.run(samples)

In [138]:
print(samples_run)

[[-4.5889136e-04  7.3938206e-02 -1.2480277e-01]
 [ 1.1469519e+00  1.9799045e+00  2.8399444e+00]]


In [149]:
import tensorflow_probability as tfp
dist = tfp.distributions.MultivariateNormalDiag(loc = mu, scale_diag = std)
print(sess.run(dist.log_prob(mu)))

[-0.4542303 -0.4542303]
